In [1]:
import os
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

def load_prelabel_output_dir(path):
    files = [os.path.join(path, i) for i in os.listdir(path) if '.json' in i]
    with open(files[0], "r") as jf:
        data = json.load(jf)
    for file in files[1:]:
        with open(file, "r") as jf:
            data_new = json.load(jf)
            data.update(data_new)
    return data

data = load_prelabel_output_dir('/data/yangxiaoran/nonrt_init/pay_rate_level_train/data/sentiment_valid_new/')

with open('./tmp/sst2/order_id_predict_pre.json','r') as jf:
    data_predict = json.load(jf)

data_p,data_n,data_m = {},{},{}
names = ['history','current','score','predict_score','predict_label']
dicts = [data_p,data_n,data_m]
for item in dicts:
    for name in names:
        if name not in item:
            item[name]=[]

for order_id in tqdm(data_predict):
    if order_id not in data:
        continue
    for group_id in data_predict[order_id]:
        if group_id not in data[order_id]:
            continue
        if data_predict[order_id][group_id]['true'] ==2:
            data_p['history'].append(data[order_id][group_id]['history'])
            data_p['current'].append(data[order_id][group_id]['current'])
            data_p['score'].append(data[order_id][group_id]['score'])
            data_p['predict_score'].append(data_predict[order_id][group_id]['predict'][2])
            data_p['predict_label'].append(np.argmax(data_predict[order_id][group_id]['predict']))
        
        if data_predict[order_id][group_id]['true'] ==0:
            data_n['history'].append(data[order_id][group_id]['history'])
            data_n['current'].append(data[order_id][group_id]['current'])
            data_n['score'].append(data[order_id][group_id]['score'])
            data_n['predict_score'].append(data_predict[order_id][group_id]['predict'][0])
            data_n['predict_label'].append(np.argmax(data_predict[order_id][group_id]['predict']))

        if data_predict[order_id][group_id]['true'] ==1:
            data_m['history'].append(data[order_id][group_id]['history'])
            data_m['current'].append(data[order_id][group_id]['current'])
            data_m['score'].append(data[order_id][group_id]['score'])
            data_m['predict_score'].append(data_predict[order_id][group_id]['predict'][1])
            data_m['predict_label'].append(np.argmax(data_predict[order_id][group_id]['predict']))

data_p = pd.DataFrame(data_p)
data_p['metric'] = data_p['score']-data_p['predict_score']
data_n = pd.DataFrame(data_n)
data_n['metric'] = -data_n['score']-data_n['predict_score']
data_m = pd.DataFrame(data_m)
data_m['metric'] = -abs(data_m['score'])-data_m['predict_score']

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5139/5139 [00:00<00:00, 377977.21it/s]


In [2]:
result_p = data_p.nlargest(30, 'metric').reset_index(drop=True)
result_p

,history,current,score,predict_score,predict_label,metric
0,0. [user]: Hey \n1. [advisor]: Hi dear \n2. [...,20. [user]: Okay thankyou. \n,2.100000,0.000020,1,2.099980
1,"0. [advisor]: Hi, welcome, this is 🌙🌟Spiritual...",18. [user]: ❤️❤️ \n19. [user]: i know \n,0.700000,0.000014,1,0.699986
2,0. [advisor]: hello \n1. [user]: Hi! \n2. [adv...,"7. [user]: Please explain, I’m new to this \n...",0.633333,0.000049,1,0.633284
3,"0. [advisor]: Hello, welcome, this is Mystic V...",13. [user]: I’m married now \n,0.633333,0.000078,1,0.633255
4,"0. [advisor]: Hi, welcome, this is 💑Enchanting...",13. [user]: Are we going to stay together \n,0.600000,0.000017,1,0.599983
5,"0. [advisor]: Hi, welcome! \n1. [advisor]: How...",13. [user]: She loves me? \n,0.600000,0.000020,1,0.599980
6,"0. [advisor]: Hi, welcome, this is Insight by ...",21. [user]: Good he’s blocked \n,0.600000,0.000021,1,0.599979
7,"0. [advisor]: Hi, welcome, this is 🔮Spiritual ...",20. [user]: They are divorcing \n,0.600000,0.000023,0,0.599977
8,"0. [advisor]: Hi, welcome, this is Insight by ...","24. [user]: I honestly I love som,one else \n...",0.600000,0.000065,1,0.599935
9,0. [user]: Hey I’m Sam 692000 \n1. [user]: 💕 \...,19. [user]: And I fell he got rid of her first...,0.600000,0.000098,1,0.599902


In [3]:
result_n = data_n.nlargest(30, 'metric').reset_index(drop=True)
result_n

,history,current,score,predict_score,predict_label,metric
0,0. [advisor]: How can I help you? \n1. [adviso...,17. [user]: How is court going to go? Will he ...,-0.800000,0.000263,1,0.799737
1,"0. [advisor]: Hi, welcome, this is Psychic Dor...",21. [user]: He loves her \n,-0.700000,0.000066,1,0.699934
2,0. [user]: Hey sis \n1. [advisor]: Hi I’m Eva💫...,24. [user]: Last time he to me l am a weirdo a...,-0.666667,0.000043,1,0.666624
3,0. [advisor]: How are you doing today dear? \n...,11. [user]: nothing is going right its just fi...,-0.666667,0.000120,1,0.666547
4,"0. [advisor]: hello,this is Joyful Energy \n1....",25. [user]: Huh \n,-0.633333,0.000018,1,0.633316
5,"0. [advisor]: Hi, I'm Spiritual Guide Jane, w...",19. [user]: Ohhhh ok yes I’ve been In severe d...,-0.633333,0.000840,1,0.632493
6,"0. [advisor]: Hi, welcome, this is Sable💫 \n1....",17. [user]: am i pregnant \n,-0.600000,0.000008,1,0.599992
7,"0. [advisor]: hello,this is Joyful Energy \n1....",27. [user]: He’s not attracted to me ? \n,-0.600000,0.000013,1,0.599987
8,0. [user]: Hi my names Krisha \n1. [advisor]: ...,12. [user]: I asked him in may if I should hav...,-0.600000,0.000013,1,0.599987
9,"0. [advisor]: I'm 🌈 Intuitive Healing,Glad to ...",12. [user]: I was very I’ll and as I started t...,-0.600000,0.000014,1,0.599986


In [4]:
result_m = data_m.nlargest(30, 'metric').reset_index(drop=True)
result_m

,history,current,score,predict_score,predict_label,metric
0,"0. [advisor]: Hi, I'm Spiritual Rose , it's a...",15. [user]: He wouldn’t change for his family ...,0.000000e+00,0.000325,0,-0.000325
1,"0. [advisor]: Hi, welcome, this is Cosmic Fusi...",23. [user]: i’m hurting mama \n,1.850372e-17,0.000357,0,-0.000357
2,"0. [advisor]: Hi, welcome, this is ⚜️Mystic Op...",23. [user]: he left me ofc \n24. [user]: but ...,3.700743e-17,0.000580,0,-0.000580
3,0. [user]: Hey I’m Sam 7132000 \n1. [user]: 💕 ...,16. [user]: ohhh \n17. [user]: So this weekend...,0.000000e+00,0.000642,2,-0.000642
4,0. [user]: Hiiii \n1. [advisor]: welcome: how ...,30. [user]: Or anything in common \n31. [user]...,9.251859e-18,0.000744,2,-0.000744
5,"0. [advisor]: Honey, what can I do for you? \n...",29. [user]: Are we going to get back together ...,0.000000e+00,0.000934,0,-0.000934
6,0. [advisor]: How can I help you? \n1. [adviso...,15. [user]: Why he ignore me and said I don't ...,0.000000e+00,0.000959,0,-0.000959
7,0. [user]: Hey I’m amber 692000 \n1. [advisor]...,22. [user]: Awww💕😭I miss him so much \n,1.850372e-17,0.001033,2,-0.001033
8,"0. [advisor]: Honey, what can I do for you? \n...",22. [user]: I miss her to so much \n,0.000000e+00,0.001380,2,-0.001380
9,"0. [user]: Hey \n1. [advisor]: Hi, welcome, th...",27. [user]: No \n28. [user]: Omg \n,0.000000e+00,0.001645,2,-0.001645


In [11]:
show_idx = 3
print('history:\n',result_p.loc[show_idx]['history'])
print('current:\n',result_p.loc[show_idx]['current'])
print('score:\n',result_p.loc[show_idx]['score'],'\n')
print('predict_socre:\n',result_p.loc[show_idx]['predict_score'],'\n')
print('predict_label:\n',result_p.loc[show_idx]['predict_label'],'\n')

history:
 0. [advisor]: Hello, welcome, this is Mystic Visionary 
1. [advisor]: Let me get your first name and date of birth please 
2. [advisor]: How can I help you? 
3. [user]: Frederick Pigram  
4. [advisor]: Date of birth please 
5. [user]: 10/23/1992.  
6. [advisor]: Thank you 
7. [user]: Would I get married and find love soon  
8. [advisor]: First things first i feel that there is lots of stress around you 
9. [advisor]: I feel that you are meant to be happy in love 
10. [advisor]: But i feel like someone or something is stopping you from your happiness 
11. [advisor]: Yes you will marry and find love soon 
12. [advisor]: I feel it’s going to be within 5 years 

current:
 13. [user]: I’m married now  

score:
 0.6333333333333333 

predict_socre:
 7.843971252441406e-05 

predict_label:
 1 



In [6]:
show_idx = 0
print('history:\n',result_n.loc[show_idx]['history'])
print('current:\n',result_n.loc[show_idx]['current'])
print('score:\n',result_n.loc[show_idx]['score'],'\n')
print('predict_socre:\n',result_n.loc[show_idx]['predict_score'],'\n')
print('predict_label:\n',result_n.loc[show_idx]['predict_label'],'\n')

history:
 0. [advisor]: How can I help you? 
1. [advisor]: please give me both your names and date of births 
2. [advisor]: please what are your questions 
3. [user]: Cordarryl and Emily 12/26/1986 10/23/1987  
4. [advisor]: okay… 
5. [user]: How are things going to turn out? 
6. [advisor]: Hold on for a moment please,  let me see through and pick up energy. 
7. [advisor]: Thank you for patience Cordarryl 
8. [user]: I’m emily 
9. [advisor]: I have seen through and picked up energy on both of you which shows he loves you but  
10. [advisor]: He seems distant at moment. Sometimes he seems like he doesn't care 
11. [advisor]: He does love you but he isn't putting efforts in the future 
12. [advisor]: He is seeming stressed and tired emotionally  
13. [user]: Is he ever going to apologize 
14. [advisor]: Yes 
15. [advisor]: He will 
16. [advisor]: And will fix all the issues sooner 

current:
 17. [user]: How is court going to go? Will he go back to prison for almost killing our son 

sco

In [7]:
show_idx = 14
print('history:\n',result_m.loc[show_idx]['history'])
print('current:\n',result_m.loc[show_idx]['current'])
print('score:\n',result_m.loc[show_idx]['score'],'\n')
print('predict_socre:\n',result_m.loc[show_idx]['predict_score'],'\n')
print('predict_label:\n',result_m.loc[show_idx]['predict_label'],'\n')

history:
 0. [advisor]: Hi, welcome, this is 🌟Mystical Maria🌟 
1. [advisor]: Okay please give me your both date of birth and good names as well 
2. [user]: Hi I’m Tiffany  
3. [advisor]: Please tell me do you have partner or have break up with him? 
4. [advisor]: Okay how may I help you Tiffany? 
5. [user]: My birthday is July 22 1994 and Payton’s is December 9 1996 we have been together since September 21 2019 but I feel like things may be bothering her and I’d like to try to save and fix this relationship  
6. [advisor]: Okay let me connect with your situation 
7. [user]: Okay  
8. [advisor]: I am seeing there are many misunderstandings in your relation 
9. [advisor]: That's why its happening and she seems cold right now 
10. [advisor]: She is not giving attention and care 
11. [advisor]: But 
12. [advisor]: But i am seeing reconciliation is possible  
13. [user]: How what do I do  
14. [advisor]: If you want to make strong connection with her then 
15. [user]: ? 
16. [advisor]: You 